In [1]:
import pyspark
import pyspark.sql.functions as fn
import pyspark.sql.types as T
from pyspark.sql import Window
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()
#demonstration DataFrame to workOn
l = [
(2020,1,"Colors"),(2020,2,"Colors"),(2020,3,"Colors"),(2020,4,"Colors"),
(2020,5,"Colors"),(2020,6,"Colors"),(2020,7,"Colors"),(2020,8,"Colors"),
    ]
df = spark.createDataFrame(l, schema=["year","week","channel"])
df.show()
df.printSchema()
df.show(truncate=False)

+----+----+-------+
|year|week|channel|
+----+----+-------+
|2020|   1| Colors|
|2020|   2| Colors|
|2020|   3| Colors|
|2020|   4| Colors|
|2020|   5| Colors|
|2020|   6| Colors|
|2020|   7| Colors|
|2020|   8| Colors|
+----+----+-------+

root
 |-- year: long (nullable = true)
 |-- week: long (nullable = true)
 |-- channel: string (nullable = true)

+----+----+-------+
|year|week|channel|
+----+----+-------+
|2020|1   |Colors |
|2020|2   |Colors |
|2020|3   |Colors |
|2020|4   |Colors |
|2020|5   |Colors |
|2020|6   |Colors |
|2020|7   |Colors |
|2020|8   |Colors |
+----+----+-------+



In [3]:
from pyspark.sql.functions import row_number
df_rn  = Window.partitionBy("week").orderBy("week")
 
df.withColumn("week_range",row_number().over(df_rn)).show()

+----+----+-------+----------+
|year|week|channel|week_range|
+----+----+-------+----------+
|2020|   7| Colors|         1|
|2020|   6| Colors|         1|
|2020|   5| Colors|         1|
|2020|   1| Colors|         1|
|2020|   3| Colors|         1|
|2020|   8| Colors|         1|
|2020|   2| Colors|         1|
|2020|   4| Colors|         1|
+----+----+-------+----------+



In [4]:
def w_week_range(part_week):
    if part_week == 1:
        return 4
    if part_week == 2:
        return 4
    if part_week == 3:
        return 4
    if part_week == 4:
        return 4
    if part_week == 5:
        return 6
    if part_week == 6:
        return 6
    if part_week == 7:
        return 8
    if part_week == 8:
        return 8
    else:
        return None

In [6]:
from pyspark.sql.functions import udf, log
test_udf = udf(lambda week : -float(week[1]), FloatType())
d = df.withColumn("new_col", test_udf(df["week"]))
df.show()

NameError: name 'FloatType' is not defined

In [ ]:
spark.udf.register("week_range", w_week_range)
 
week_udf = udf(w_week_range, LongType())
 
spark.range(1,17).createOrReplaceTempView("test4")
 
week_df_r = spark.table("test4")
 
df_week_test = df_rn.withColumn("week_range", week_udf("part_week"))
df_week_test.show()